# CNN From Ch.12 using ShallowNet

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# checks to see if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pr
print("CUDA Available: ", torch.cuda.is_available())

CUDA Available:  True
